In [1]:
#数据预处理
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import LabelEncoder
from dummyPy import OneHotEncoder
import random
import pickle  # 存储临时变量

## 读文件
#file_path = '/home/admin/avazu/'
fp_train = 'train.csv'
fp_test  = 'test.csv'

## 下采样写文件
fp_sub_train_f = 'sub_train_f.csv'
fp_col_counts = 'col_counts'

## data after selecting features (LR_fun needed)
## and setting rare categories' value to 'other' (feature filtering)
fp_train_f = 'train_f.csv'
fp_test_f  = 'test_f.csv'

## 存储标签编码和one-hot编码
fp_lb_enc = 'lb_enc'
fp_oh_enc = 'oh_enc'

##==================== 数据预处理 ====================##
## 特征选择
cols = ['C1', 
        'banner_pos', 
        'site_id',
        'site_category',
        'app_id',
        'app_category', 
        'device_id',
        'device_model',
        'device_type', 
        'device_conn_type',
        'C14', 
        'C15',
        'C16',
        'C17', 
        'C18',
        'C19',
        'C20']

cols_train = ['id', 'click']
cols_test  = ['id']
cols_train.extend(cols)
cols_test.extend(cols)

## 数据加载
print('loading data...')
df_train_ini = pd.read_csv(fp_train, nrows = 10)
df_train_org = pd.read_csv(fp_train, chunksize = 1000000, iterator = True)
df_test_org  = pd.read_csv(fp_test,  chunksize = 1000000, iterator = True)

#----- 统计分类变量 数值个数 -----#
## 初始化
cols_counts = {}  # 统计每个特征的分类数量
for col in cols:
    cols_counts[col] = df_train_ini[col].value_counts()

## 统计训练集
for chunk in df_train_org:
    for col in cols:
        cols_counts[col] = cols_counts[col].append(chunk[col].value_counts())

## 统计测试集
for chunk in df_test_org:
    for col in cols:
        cols_counts[col] = cols_counts[col].append(chunk[col].value_counts())
        
## 统计
for col in cols:
    cols_counts[col] = cols_counts[col].groupby(cols_counts[col].index).sum()
    # sort the counts
    cols_counts[col] = cols_counts[col].sort_values(ascending=False)   

## 存储value_counting
pickle.dump(cols_counts, open(fp_col_counts, 'wb'))
'''
## 绘制分布
fig = plt.figure(1)
for i, col in enumerate(cols):
    ax = fig.add_subplot(4, 3, i+1)
    ax.fill_between(np.arange(len(cols_counts[col])), cols_counts[col].get_values())
    # ax.set_title(col)
plt.show()
'''
## 只保存前K个分类变量
k = 99
col_index = {}
for col in cols:
    col_index[col] = cols_counts[col][0: k].index

df_train_org = pd.read_csv(fp_train, dtype = {'id': str}, chunksize = 1000000, iterator = True)
df_test_org  = pd.read_csv(fp_test,  dtype = {'id': str}, chunksize = 1000000, iterator = True)

## 训练集
hd_flag = True  # add column names at 1-st row
for chunk in df_train_org:
    df = chunk.copy()
    for col in cols:
        df[col] = df[col].astype('object')
        # assign all the rare variables as 'other'
        df.loc[~df[col].isin(col_index[col]), col] = 'other'
    with open(fp_train_f, 'a') as f:
        df.to_csv(f, columns = cols_train, header = hd_flag, index = False)
    hd_flag = False

## 测试集
hd_flag = True  # 第一个chunk需要有header
for chunk in df_test_org:
    df = chunk.copy()
    for col in cols:
        df[col] = df[col].astype('object')
        # 设置其他不常用变量为other
        df.loc[~df[col].isin(col_index[col]), col] = 'other'
    with open(fp_test_f, 'a') as f:
        df.to_csv(f, columns = cols_test, header = hd_flag, index = False)      
    hd_flag = False    

## 对分类变量进行标签编码
lb_enc = {}
for col in cols:
    col_index[col] = np.append(col_index[col], 'other')

for col in cols:
    lb_enc[col] = LabelEncoder()
    lb_enc[col].fit(col_index[col])
    
## 存储标签编码
pickle.dump(lb_enc, open(fp_lb_enc, 'wb'))

## one-hot编码
oh_enc = OneHotEncoder(cols)

df_train_f = pd.read_csv(fp_train_f, index_col=None, chunksize=500000, iterator=True)
df_test_f  = pd.read_csv(fp_test_f, index_col=None, chunksize=500000, iterator=True)

for chunk in df_train_f:
    oh_enc.fit(chunk)
for chunk in df_test_f:
    oh_enc.fit(chunk)
    
## 存储one-hot编码
pickle.dump(oh_enc, open(fp_oh_enc, 'wb'))


# 计算总训练样本 约46M
n = sum(1 for line in open(fp_train_f)) - 1 
# 保存下采样训练样本 2M
s = 2000000

## 设置哪些行不需要读 skip，不需要读的行数为n-s
skip = sorted(random.sample(range(1, n+1), n-s)) 
df_train = pd.read_csv(fp_train_f, skiprows = skip)
df_train.columns = cols_train

## 存储下采样的结果
df_train.to_csv(fp_sub_train_f, index=False) 



loading data...


In [1]:
# 使用DCN模型对Avazu CTR进行预估
import pandas as pd
import numpy as np
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt 
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from deepctr.models import DCN #使用Deep & Cross
from deepctr.inputs import SparseFeat,get_feature_names
import pickle


##==================== 设置文件路径File-Path (fp) ====================##
#file_path = '/home/admin/avazu/'
fp_train_f = "sub_train_f.csv" #使用小样本进行训练

##==================== DCN 训练 ====================##
data = pd.read_csv(fp_train_f, dtype={'id':str}, index_col=None)
print('data loaded')

#数据加载
sparse_features = ['C1', 
        'banner_pos', 
        'site_id',
        'site_category',
        'app_id',
        'app_category', 
        'device_id',
        'device_model',
        'device_type', 
        'device_conn_type',
        'C14', 
        'C15',
        'C16',
        'C17', 
        'C18',
        'C19',
        'C20']
target = ['click']

# 对特征标签进行编码
for feature in sparse_features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature])
# 计算每个特征中的 不同特征值的个数
fixlen_feature_columns = [SparseFeat(feature, data[feature].nunique()) for feature in sparse_features]
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)
#print(fixlen_feature_columns)
#print(feature_names)

# 将数据集切分成训练集和测试集
train, test = train_test_split(data, test_size=0.2)
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

# 使用DCN进行训练
#model = DCN(linear_feature_columns, dnn_feature_columns, task='regression')
model = DCN(linear_feature_columns, dnn_feature_columns, task='binary')
model.compile("adam", "binary_crossentropy", metrics=['binary_crossentropy'], )


history = model.fit(train_model_input, train[target].values, batch_size=256, epochs=10, verbose=True, validation_split=0.2, )
# 使用DCN进行预测
pred_ans = model.predict(test_model_input, batch_size=256)
# 输出RMSE或MSE
mse = round(mean_squared_error(test[target].values, pred_ans), 4)
rmse = mse ** 0.5
print("test RMSE", rmse)

# 输出LogLoss
from sklearn.metrics import log_loss
score = log_loss(test[target].values, pred_ans)
print("LogLoss", score)

data loaded
Train on 1280000 samples, validate on 320000 samples
Epoch 1/10


D:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
D:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


1280000/1280000 [==============================] - 47s 37us/sample - loss: 0.4071 - binary_crossentropy: 0.4069 - val_loss: 0.4033 - val_binary_crossentropy: 0.4030
Epoch 2/10
1280000/1280000 [==============================] - 40s 32us/sample - loss: 0.4030 - binary_crossentropy: 0.4026 - val_loss: 0.4013 - val_binary_crossentropy: 0.4008
Epoch 3/10
1280000/1280000 [==============================] - 39s 31us/sample - loss: 0.4017 - binary_crossentropy: 0.4011 - val_loss: 0.4008 - val_binary_crossentropy: 0.4002
Epoch 4/10
1280000/1280000 [==============================] - 39s 31us/sample - loss: 0.4008 - binary_crossentropy: 0.4002 - val_loss: 0.4012 - val_binary_crossentropy: 0.4005
Epoch 5/10
1280000/1280000 [==============================] - 38s 30us/sample - loss: 0.4001 - binary_crossentropy: 0.3994 - val_loss: 0.4003 - val_binary_crossentropy: 0.3996
Epoch 6/10
1280000/1280000 [==============================] - 37s 29us/sample - loss: 0.3997 - binary_crossentropy: 0.3990 - val_lo

In [2]:
from deepctr.models import xDeepFM

model = xDeepFM(linear_feature_columns, dnn_feature_columns, task='binary')
model.compile("adam", "binary_crossentropy", metrics=['binary_crossentropy'], )


history = model.fit(train_model_input, train[target].values, batch_size=256, epochs=10, verbose=True, validation_split=0.2, )
# 使用xDeepFM进行预测
pred_ans = model.predict(test_model_input, batch_size=256)
# 输出RMSE或MSE
mse = round(mean_squared_error(test[target].values, pred_ans), 4)
rmse = mse ** 0.5
print("test RMSE", rmse)

# 输出LogLoss
score = log_loss(test[target].values, pred_ans)
print("LogLoss", score)

Train on 1280000 samples, validate on 320000 samples
Epoch 1/10


D:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
D:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


1280000/1280000 [==============================] - 216s 168us/sample - loss: 0.4068 - binary_crossentropy: 0.4066 - val_loss: 0.4041 - val_binary_crossentropy: 0.4038
Epoch 2/10
1280000/1280000 [==============================] - 215s 168us/sample - loss: 0.4027 - binary_crossentropy: 0.4023 - val_loss: 0.4016 - val_binary_crossentropy: 0.4011
Epoch 3/10
1280000/1280000 [==============================] - 214s 167us/sample - loss: 0.4011 - binary_crossentropy: 0.4006 - val_loss: 0.4007 - val_binary_crossentropy: 0.4002
Epoch 4/10
1280000/1280000 [==============================] - 217s 170us/sample - loss: 0.4003 - binary_crossentropy: 0.3997 - val_loss: 0.4003 - val_binary_crossentropy: 0.3997
Epoch 5/10
1280000/1280000 [==============================] - 210s 164us/sample - loss: 0.3995 - binary_crossentropy: 0.3988 - val_loss: 0.3998 - val_binary_crossentropy: 0.3991
Epoch 6/10
1280000/1280000 [==============================] - 215s 168us/sample - loss: 0.3990 - binary_crossentropy: 0.3

In [10]:
import xgboost as xgb

param = {'boosting_type':'gbdt',
                         'objective' : 'binary:logistic', 
                         'eval_metric' : 'logloss',
                         'eta' : 0.01,
                         'max_depth' : 15,
                         'colsample_bytree':0.8,
                         'subsample': 0.9,
                         'subsample_freq': 8,
                         'alpha': 0.6,
                         'lambda': 0,
        }

X_train, X_valid, y_train, y_valid = train_test_split(train.drop(['id','click'],axis=1), train['click'], test_size=0.2)

train_data = xgb.DMatrix(X_train, label=y_train)
valid_data = xgb.DMatrix(X_valid, label=y_valid)
test_data = xgb.DMatrix(test.drop(['id','click'],axis=1))

model_xgb = xgb.train(param, train_data, evals=[(train_data, 'train'), (valid_data, 'valid')], num_boost_round = 10000, early_stopping_rounds=200, verbose_eval=25)
predict = model_xgb.predict(test_data)

# 输出RMSE或MSE
mse = round(mean_squared_error(test[target].values, predict), 4)
rmse = mse ** 0.5
print("test RMSE", rmse)

# 输出LogLoss
score = log_loss(test[target].values, predict)
print("LogLoss", score)

[13:06:49] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.0.0\src\learner.cc:328: 
Parameters: { boosting_type, subsample_freq } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-logloss:0.68824	valid-logloss:0.68825
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 200 rounds.
[25]	train-logloss:0.59065	valid-logloss:0.59164
[50]	train-logloss:0.52771	valid-logloss:0.52960
[75]	train-logloss:0.48555	valid-logloss:0.48833
[100]	train-logloss:0.45696	valid-logloss:0.46052
[125]	train-logloss:0.43703	valid-logloss:0.44145
[150]	train-logloss:0.42300	valid-logloss:0.42824
[175]	train-logloss:0.41302	valid-logloss:0.41914
[200]	train-logloss:0.40592	valid-logloss:0.41

In [15]:
test_sub = pd.read_csv('test_f.csv', dtype={'id':str})
for feature in sparse_features:
    lbe = LabelEncoder()
    test_sub[feature] = lbe.fit_transform(test_sub[feature])

In [19]:
predict_data = xgb.DMatrix(test_sub.drop(['id'],axis=1))
predict_xgb = model_xgb.predict(predict_data)

In [20]:
submission = pd.read_csv('sampleSubmission.csv', dtype={'id':str})
submission['click_xgb'] = predict_xgb

In [22]:
predict_model_input = {name:test_sub[name].values for name in feature_names}
predict_xdeepfm = model.predict(predict_model_input, batch_size=256)
submission['click_xdeepfm'] = predict_xdeepfm

In [23]:
submission['click'] = (submission['click_xdeepfm']+submission['click_xgb'])/2

In [24]:
print(submission)

                           id     click  click_xgb  click_xdeepfm
0        10000174058809263569  0.057438   0.108411       0.006465
1        10000182526920855428  0.024427   0.048549       0.000305
2        10000554139829213984  0.036340   0.072475       0.000204
3        10001094637809798845  0.009916   0.017265       0.002567
4        10001377041558670745  0.026496   0.045275       0.007716
5        10001521204153353724  0.143997   0.202357       0.085636
6        10001911056707023378  0.017299   0.034508       0.000090
7        10001982898844213216  0.006131   0.012223       0.000038
8        10002000217531288531  0.023785   0.040772       0.006797
9        10002107385290585663  0.024664   0.048959       0.000369
10       10002121728305927027  0.034007   0.067972       0.000043
11       10002246330438763746  0.024813   0.043304       0.006322
12       10002274319697022546  0.018790   0.036070       0.001510
13       10002614093820007721  0.013782   0.006283       0.021280
14       1

In [25]:
submission = submission.drop(['click_xdeepfm','click_xgb'],axis =1)

In [27]:
submission.to_csv('submission.csv',index = 0)